# <b>Required Imports</b>

In [42]:
import pickle,sys,os, time

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()

print(tf.__version__)

2.7.0


In [43]:
!wget https://github.com/GastonMazzei/TraceMeOut/blob/main/processed_trace/Dataset0.pkl?raw=true
!wget https://github.com/GastonMazzei/TraceMeOut/blob/main/processed_trace/Dataset1.pkl?raw=true
!wget https://github.com/GastonMazzei/TraceMeOut/blob/main/processed_trace/Dataset2.pkl?raw=true
!wget https://github.com/GastonMazzei/TraceMeOut/blob/main/processed_trace/Dataset3.pkl?raw=true

--2021-12-31 00:35:04--  https://github.com/GastonMazzei/TraceMeOut/blob/main/processed_trace/Dataset0.pkl?raw=true
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/GastonMazzei/TraceMeOut/raw/main/processed_trace/Dataset0.pkl [following]
--2021-12-31 00:35:05--  https://github.com/GastonMazzei/TraceMeOut/raw/main/processed_trace/Dataset0.pkl
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/GastonMazzei/TraceMeOut/main/processed_trace/Dataset0.pkl [following]
--2021-12-31 00:35:05--  https://raw.githubusercontent.com/GastonMazzei/TraceMeOut/main/processed_trace/Dataset0.pkl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubuser

In [44]:
!ls

'Dataset0.pkl?raw=true'    'Dataset1.pkl?raw=true.3'  'Dataset2.pkl?raw=true.6'
'Dataset0.pkl?raw=true.1'  'Dataset1.pkl?raw=true.4'  'Dataset3.pkl?raw=true'
'Dataset0.pkl?raw=true.2'  'Dataset1.pkl?raw=true.5'  'Dataset3.pkl?raw=true.1'
'Dataset0.pkl?raw=true.3'  'Dataset1.pkl?raw=true.6'  'Dataset3.pkl?raw=true.2'
'Dataset0.pkl?raw=true.4'  'Dataset2.pkl?raw=true'    'Dataset3.pkl?raw=true.3'
'Dataset0.pkl?raw=true.5'  'Dataset2.pkl?raw=true.1'  'Dataset3.pkl?raw=true.4'
'Dataset0.pkl?raw=true.6'  'Dataset2.pkl?raw=true.2'  'Dataset3.pkl?raw=true.5'
'Dataset1.pkl?raw=true'    'Dataset2.pkl?raw=true.3'  'Dataset3.pkl?raw=true.6'
'Dataset1.pkl?raw=true.1'  'Dataset2.pkl?raw=true.4'   sample_data
'Dataset1.pkl?raw=true.2'  'Dataset2.pkl?raw=true.5'


In [45]:
datas = {}
for n in [0,1,2,3]:
  with open(f'Dataset{n}.pkl?raw=true','rb') as f:
    datas[n] = pickle.load(f)

#<b>Configuration File: architecture & hyperparameters</b>

(<i>If the datasets change in the repo, non-architectural information should be updated using the configuration.py file. Most probably an error will be raised because of size mismatch if this is not updated.</i>)

In [46]:
# Model-specific parameters
T=8 # duration of the window in dt units
dt = 4000 # time in microseconds
UNIQUES=3807  #number of unique ids
MI=4762  #max number of interactions
ML=4765  #max number of leaves
NCATEGORIES=2

# Architectural parameters
ACT1 = 'relu'
FILTERS1 = 8
KSIZE1 = (2,1)
PSIZE1 = (max([T//4,2]),)
NDENSE1 = 8
DROP1 = 0.4

ACT2 = 'relu'
FILTERS2 = 8
KSIZE2 = (2,2)
PSIZE2 = (max([T//2,2]),1)
stride = (1,1)
NDENSE2 = 8
DROP2 = 0.5


ACT4='relu'
NDENSE4=8
DROP4 = 0.3


ACT3='relu'
NDENSE3=8
DROP3 = 0.3


# Training parameters
VAL=0.25
BATCH=128
EPOCHS=50
L=5 # a length used to generate random data just for testing
LR=0.05
SAMPLES = 30


# Extras
POOLING = False
PROCS=[3, 2, 0, 1]

In [59]:
# Assert the dataset's keys and the procs in the configuration are the same :-)
temp1, temp2 = sorted(PROCS), sorted(list(datas.keys()))
assert(temp1 == temp2)
NPROCS = len(PROCS)

# define two sets of inputs: 
MAXPAD = max([MI,ML])
input_shape_flavours = (BATCH, T, MAXPAD, 1)
input_shape_structure = (BATCH, T, MAXPAD, 2)

def produce_model():
	inputFlavours = tf.keras.Input(shape=input_shape_flavours[1:])
	inputStructure = tf.keras.Input(shape=input_shape_structure[1:])

	# the first branch operates on the first input (https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv1D)
	x = tf.keras.layers.Conv2D(
				# Filters, Kersize, Strides, Padding,  Activation
				FILTERS1,         KSIZE1,       (1,1),      'valid',  activation = ACT1,
				input_shape = input_shape_flavours[1:]
				)(inputFlavours)
	x = tf.keras.layers.Conv2D(
				# Filters, Kersize, Strides, Padding,  Activation
				FILTERS1 * 2,         KSIZE1,       (1,1),      'valid',  activation = ACT1,
				)(x)
	x = tf.keras.layers.Conv2D(
				# Filters, Kersize, Strides, Padding,  Activation
				FILTERS1 * 2,         KSIZE1,       (1,1),      'valid',  activation = ACT1,
				)(x)
	#x = tf.keras.layers.MaxPool1D(pool_size=PSIZE1)(x)
	x = tf.keras.layers.Flatten()(x)
	x = tf.keras.layers.Dropout(DROP1)(x)
	x = tf.keras.layers.Dense(NDENSE1, activation = ACT1)(x)
	x = tf.keras.layers.Dropout(DROP1)(x)
	x = tf.keras.layers.Dense(NDENSE1 // 2, activation = ACT1)(x)
	x = tf.keras.layers.Dropout(DROP1)(x)
	x = tf.keras.layers.Dense(NDENSE1 // 2 // 2, activation = ACT1)(x)
	x = tf.keras.Model(inputs = inputFlavours, outputs=x)

	# the second branch opreates on the second input (https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D)
	y = tf.keras.layers.Conv2D(
				# Filters, Kersize, Strides, Padding,  Activation
				FILTERS2,         KSIZE2,    stride,      'valid',  activation = ACT2,
				input_shape = input_shape_structure[1:]
				)(inputStructure)
	y = tf.keras.layers.Conv2D(
				# Filters, Kersize, Strides, Padding,  Activation
				FILTERS2 * 2,         KSIZE2,       stride,      'valid',  activation = ACT2,
				)(y)
	#y = tf.keras.layers.MaxPool2D(pool_size=PSIZE2)(y)
	y = tf.keras.layers.Conv2D(
				# Filters, Kersize, Strides, Padding,  Activation
				FILTERS2 * 2,         KSIZE2,       stride,      'valid',  activation = ACT2,
				)(y)
	y = tf.keras.layers.MaxPool2D(pool_size=PSIZE2)(y)
	y = tf.keras.layers.Flatten()(y)
	y = tf.keras.layers.Dropout(DROP2)(y)
	y = tf.keras.layers.Dense(NDENSE2, activation = ACT2)(y)
	y = tf.keras.layers.Dropout(DROP2)(y)
	y = tf.keras.layers.Dense(NDENSE2 // 2, activation = ACT2)(y)
	y = tf.keras.layers.Dropout(DROP2)(y)
	y = tf.keras.layers.Dense(NDENSE2 // 2 // 2, activation = ACT2)(y)
	y = tf.keras.Model(inputs = inputStructure, outputs=y)



	# combine the output of the two branches
	combined = tf.keras.layers.concatenate([x.output, y.output])
	z = tf.keras.layers.Dense(NDENSE3, activation = ACT3)(combined)
	z = tf.keras.layers.Dropout(DROP3)(z)
	z = tf.keras.layers.Dense(NDENSE3, activation = ACT3)(z)
	z = tf.keras.layers.Dropout(DROP3)(z)
	z = tf.keras.layers.Dense(NDENSE3, activation = ACT3)(z)

	# our model will accept the inputs of the two branches and
	# then output a single value
	return tf.keras.Model(inputs=[x.input, y.input], outputs=z)

models = []
for n in range(NPROCS):
	models += [produce_model()]
TOTALINPUTS = []
for m in models:
	TOTALINPUTS += [m.inputs[0], m.inputs[1]]
combined = tf.keras.layers.concatenate([m.output for m in models])
w = tf.keras.layers.Dense(NDENSE4, activation = ACT4)(combined)
w = tf.keras.layers.Dropout(DROP4)(w)
w = tf.keras.layers.Dense(NDENSE4, activation = ACT4)(combined)
w = tf.keras.layers.Dropout(DROP4)(w)
w = tf.keras.layers.Dense(NDENSE4, activation = ACT4)(combined)
w = tf.keras.layers.Dense(NCATEGORIES, activation="softmax")(w)
model = tf.keras.Model(inputs=TOTALINPUTS, outputs=w)

# Compile the model :-)
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=LR),
							loss=tf.keras.losses.CategoricalCrossentropy(),
							metrics=[
											#tf.keras.metrics.CategoricalCrossentropy(),
											tf.keras.metrics.CategoricalAccuracy(),
											#tf.keras.metrics.AUC(),
											])

# Print input size
print(model.summary())
IShape = model.input_shape
OShape = model.output_shape

Model: "model_38"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_18 (InputLayer)          [(None, 8, 4765, 2)  0           []                               
                                ]                                                                 
                                                                                                  
 input_20 (InputLayer)          [(None, 8, 4765, 2)  0           []                               
                                ]                                                                 
                                                                                                  
 input_22 (InputLayer)          [(None, 8, 4765, 2)  0           []                               
                                ]                                                          

#<b>Building the dataset generators to feed the Neural Net</b>

In [48]:
def trimmer(datas):
  l = []
  for n in datas.keys():
    l += [len(datas[n]['X1'])]
  prune_length = min([len(d['X1']) for d in datas.values()])
  for n in datas.keys():
    new = {}
    for k in datas[n].keys():
      new[k] = datas[n][k][:prune_length]
    datas[n] = new.copy()
  return

def recomputer(L):  
  global BATCH
  LTR = int(L*(1-VAL))
  print(f"Old batch size was: {BATCH}")
  B = 32
  LTR = LTR//B * B # LTR is approximated to the closest multiple of B :-) which is convenient for GPU's given their architecture
  LVA = L - LTR
  LVA = LVA // B * B # THe same for LVA
  BATCH = (BATCH // B + 1) * B # And the same for Batch Size
  print(f"New batch size is: {BATCH}, LVA and LTR are: {LVA}, {LTR} and L is {L}, ... does L>=LTR+LVA? {L>=(LTR+LVA)}")
  return BATCH, L, LVA, LTR


def get_refs_for(data):
  global T
  global SAMPLES
  y = np.asarray(data['Y'])
  ixs = np.asarray(range(len(y)))
  pos = ixs[y==1]
  neg = ixs[y==0]

  minL = min([len(pos), len(neg)])

  # minL/(T+SAMPLES)=SLICES
  WIDTH = (T+SAMPLES)
  SLICES = minL//WIDTH
  assert(SLICES>0)

  # Produce the slices
  posSlices = [pos[i*WIDTH : (i+1)*WIDTH] for i in range(SLICES)]
  negSlices = [neg[i*WIDTH : (i+1)*WIDTH] for i in range(SLICES)]

  # Filter only connected slices :-)
  TARGET = [0] * (T+SAMPLES-1)
  posSlices = [x for x in posSlices if (np.diff(x)-1).tolist() == TARGET]
  negSlices = [x for x in negSlices if (np.diff(x)-1).tolist() == TARGET]

  # Enforce a tolerance of e.g. 3% difference in length, i.e. how unbalanced the dataset will be.
  TOL = max([int(0.03*len(posSlices)),1])
  assert(abs(len(posSlices) - len(negSlices)) <= TOL)

  # Produce more effective indexes
  posRefs = []
  for x in posSlices:
    posRefs += [x[i+T] for i in range(SAMPLES)]
  negRefs = []
  for x in negSlices:
    negRefs += [x[i+T] for i in range(SAMPLES)]

  # Shuffle and return :-)
  np.random.shuffle(negRefs)
  np.random.shuffle(posRefs)
  return negRefs, posRefs


def build_data(datas,n):
  global MAXPAD
  # Open data
  data = datas[n]
  X1, X20, Y = data['X1'],data['X2'],data['Y']

  # Process Y
  ONE_HOT_Y = np.zeros((len(Y),2))
  for i in range(len(Y)):
      ONE_HOT_Y[i,Y[i]] = 1
  ONE_HOT_Y = ONE_HOT_Y.astype('float32')

  # Process X1
  X1_TRACKER = []
  for i_,X_ in enumerate(X1):
      X1_TRACKER.append((MAXPAD-len(X_)))
      X1[i_] = [float(x) / UNIQUES for x in X_]

  # Process X2
  X2 = [[] for _ in range(len(X20))]
  X2_TRACKER = []
  for i,X_ in enumerate(X20):
      c = 0
      for i_,y_ in enumerate(X_):
          for z in y_:
              X2[i] += [[float(i_)/MAXPAD,float(z)/MAXPAD]]
              c += 1
      X2_TRACKER.append(MAXPAD-c) 

  # Record the results
  datas[n]['X1'] = (X1, X1_TRACKER)
  datas[n]['X2'] = (X2, X2_TRACKER)
  datas[n]['Y'] = (ONE_HOT_Y,)

  return


def produce_data(A,B,Refs):
    counter = 0  
    w = list(range(A,B))
    np.random.shuffle(w)
    YS = datas[PROCS[0]]['Y'][0]
    LIMIT  = 2 * len(Refs[0])
    while counter < LIMIT:
            i = Refs[counter % 2][A + counter // 2]
            yield (
                    tuple([
                  ( tf.convert_to_tensor(np.asarray([ (datas[nproc]['X1'][0][j] + [0.] * datas[nproc]['X1'][1][j]) for j in range(i-T+1,i+1)]).reshape(1,T,-1,1)), 
                tf.convert_to_tensor(np.asarray([ (datas[nproc]['X2'][0][j] + [[0., 0.]] * datas[nproc]['X2'][1][j]) for j in range(i-T+1,i+1)]).reshape(1,T,-1,2)),
              ) for nproc in range(len(PROCS))
                        ]), 
                    tf.convert_to_tensor(YS[i:i+1,:].reshape(1,2)),
                  )
            counter += 1
            

trimmer(datas)
Refs  = get_refs_for(datas[1])
BATCH, L, LVA, LTR = recomputer(len(Refs[0]))
print(f'Len of refs is {len(Refs[0])}, L,LVA,LTR are {L},{LVA},{LTR}')
for n in PROCS:
  build_data(datas,n)


def produce_entire_datasets(L0,Lref, name=''):
  """
  This function attempts to fit the entire dataset into memory.
  For a 30-secs kernel trace, approx 10 Mb of pickle data per processor, it fails.
  The key is that it includes padding ;-)
  """
  global Refs
  it = produce_data(L0,Lref, Refs)
  data = []
  counter = 0
  while True:
    try:
      data += [it.__next__()]
    except:
      break
    counter += 1
  return data


def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))): # if value ist tensor
        value = value.numpy() # get value of tensor
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

FEATURE_NAME = []
for i in range(len(PROCS)):
  FEATURE_NAME += [f"X0_P{i}", f"X1_P{i}"]
FEATURE_NAME += ["Y"]
parse_dic = {
    fname: tf.io.FixedLenFeature([], tf.string) for fname in FEATURE_NAME
    }
def _parse_tfr_element(element):
  example_message = tf.io.parse_single_example(element, parse_dic)
  byteFeatures = [example_message.get(FEATURE_NAME[i],[]) for i in range(BASE)]
  features = [tf.io.parse_tensor(x, out_type=tf.float32) for x in byteFeatures]
  return features    

Old batch size was: 128
New batch size is: 160, LVA and LTR are: 448, 1344 and L is 1800, ... does L>=LTR+LVA? True
Len of refs is 1800, L,LVA,LTR are 1800,448,1344


#<b>Training the Neural Net</b>

In [159]:
DATASET_ITERATOR_APPROACH = [True,False][1]

In [ ]:
if DATASET_ITERATOR_APPROACH:
  OS = (
          tuple([
            (tf.TensorSpec(shape=(None,T,MAXPAD,1), dtype=tf.float32),
            tf.TensorSpec(shape=(None,T,MAXPAD,2), dtype=tf.float32)
            ) for nproc in range(len(PROCS)) 
            ]),
            tf.TensorSpec(shape=(None,NCATEGORIES), dtype=tf.float32),
      )
  print(f'Finished building the output structure')
  trainD = tf.data.Dataset.from_generator(lambda: produce_data(0,LTR, Refs), output_signature=OS)#output_types=(tf.float32), output_shapes=OS)
  print(f'Finished building the training dataset')
  valD = tf.data.Dataset.from_generator(lambda: produce_data(LTR,L, Refs), output_signature=OS)# output_types=(tf.float32), output_shapes=OS)
  print(f'Finished building the validation dataset')
  print(f'About to train! :-)')
  history = model.fit(trainD, epochs=10, batch_size=BATCH, validation_data=valD, verbose=2)
  print('Finished training! :-)')

In [162]:
if not DATASET_ITERATOR_APPROACH:

  # Spawn data iterators
  trainD = produce_data(0,LTR, Refs)
  valD = produce_data(LTR,L, Refs)
    
  # Prepare variables, configuration, containers, and write the entire
  # dataset to a file that can be progressively piped later. 
  # The point is that TensorFlow can prefetch and/or use parallelization
  # to loop more efficiently, as opposed to the GIL impacting on the 
  # performance of the dataset from generator in the typical training :-)
  train_file_paths = [f'data.tfrecords-train{i}' for i in range(4)]
  THR_TRAINF = LTR//4
  val_file_paths = [f'data.tfrecords-val{i}' for i in range(4)]
  THR_VALF = (L-LTR)//4
  BASE = 2 * len(PROCS) + 1

  TEST = True
  KEEP_GOING = True
  counter = 0
  PERIOD1 = 50
  data = []
  nameix = 0
  while KEEP_GOING:
    if (counter+1)%THR_TRAINF == 0:
      if nameix==len(train_file_paths)-1: pass
      else: nameix += 1
    try:
      data += [trainD.__next__()]
      if TEST:
        if counter > 30: raise Exception("This is only a test ;-)")
    except:
      KEEP_GOING = False
      counter = PERIOD1-1
    counter += 1
    if counter % PERIOD1 == 0:
      arrays = []
      for dat in data:
        local_array = []
        for P in dat[0]:
          local_array += [tf.io.serialize_tensor(P[0].astype('float32')), tf.io.serialize_tensor(P[1].astype('float32'))]
        local_array += [tf.io.serialize_tensor(dat[1])]
        arrays.append(local_array.copy())
      with tf.io.TFRecordWriter(train_file_paths[nameix]) as writer:
        for serialized_arrays in arrays:
          features = {FEATURE_NAME[c]: _bytes_feature(serialized_array) for c,serialized_array in enumerate(serialized_arrays)}
          example_message = tf.train.Example(features=tf.train.Features(feature=features))
          writer.write(example_message.SerializeToString())
      del data
      data = []
  data = []
  KEEP_GOING = True
  counter = 0
  nameix = 0
  while KEEP_GOING:
    if (counter+1)%THR_VALF == 0:
      if nameix==len(val_file_paths)-1: pass
      else: nameix += 1
    try:
      data += [valD.__next__()]
      if TEST:
        if counter > 30: raise Exception("This is only a test ;-)")
    except:
      KEEP_GOING = False
      counter = PERIOD1-1
    counter += 1
    if counter % PERIOD1 == 0:
      arrays = []
      for dat in data:
        local_array = []
        for P in dat[0]:
          local_array += [tf.io.serialize_tensor(P[0].astype('float32')), tf.io.serialize_tensor(P[1].astype('float32'))]
        local_array += [tf.io.serialize_tensor(dat[1])]
        arrays.append(local_array.copy())
      with tf.io.TFRecordWriter(val_file_paths[nameix]) as writer:
        for serialized_arrays in arrays:
          features = {FEATURE_NAME[c]: _bytes_feature(serialized_array) for c,serialized_array in enumerate(serialized_arrays)}
          example_message = tf.train.Example(features=tf.train.Features(feature=features))
          writer.write(example_message.SerializeToString())
      del data
      data = []      

In [166]:
if not DATASET_ITERATOR_APPROACH:

  print(f'About to train! :-)') 

  tfr_dataset = tf.data.TFRecordDataset(train_file_paths, num_parallel_reads=4) 
  dataset = tfr_dataset.map(_parse_tfr_element)
  tfr_val_dataset = tf.data.TFRecordDataset(val_file_paths, num_parallel_reads=4) 
  val_dataset = tfr_dataset.map(_parse_tfr_element)
  for E in range(EPOCHS):
    t0 = time.time()
    Y = []
    for i,x in enumerate(dataset):
      a,b,c,d,e,f,g,h,k = x
      # We can still add a layer of control by saving the data with their batches and all :-) UPGRADE
      if i==0:
        X0a = a
        X0b = b
        X1a = c
        X1b = d
        X2a = e
        X2b = f
        X3a = g
        X3b = h
      else:
        X0a = tf.concat([X0a,a],0)
        X0b = tf.concat([X0b,b],0)
        X1a = tf.concat([X1a,c],0)
        X1b = tf.concat([X1b,d],0)
        X2a = tf.concat([X2a,e],0)
        X2b = tf.concat([X2b,f],0)
        X3a = tf.concat([X3a,g],0)
        X3b = tf.concat([X3b,h],0)
      Y.append(k[0])
      if (i+1)%B==0:
        xdata = [X0a,X0b,X1a,X1b,X2a,X2b,X3a,X3b]
        with tf.GradientTape() as tape:
            logits = model(xdata, training=True) 
            loss_value = model.loss(Y, logits)
        grads = tape.gradient(loss_value, model.trainable_weights)
        model.optimizer.apply_gradients(zip(grads, model.trainable_weights))

    for i,x in enumerate(dataset):
      # Collect data with a mechanism as in the previous section 
      #logits = model(xdata, training=True)
      #model.metrics.compute or something
      # Extra bibliography is: https://www.tensorflow.org/guide/keras/writing_a_training_loop_from_scratch
      pass

    # Time report... is this method really better?   
    tf = time.time()
    print(f'Time for Epoch {E} and batch size {B} has been {tf-t0} seconds')

  print('Finished training! :-)')

About to train! :-)
Finished training! :-)


#<b>Plotting the Neural Net results</b>

In [ ]:
if DATASET_ITERATOR_APPROACH:
  # Display results
  f,ax = plt.subplots(1,2,figsize=(15,10))

  ax[0].plot(history.history['loss'],label='loss', c='k', lw=2)
  ax[0].plot(history.history['val_loss'], label='val loss', c='r', lw=2)
  ax[0].grid()
  ax[0].set_title('Training and Validation Loss* over the Epochs\n*TensorFlow categorical crossentropy')
  ax[0].set_ylim(0,None)
  ax[0].legend()

  ax[1].plot(history.history['categorical_accuracy'],label='accuracy', c='k', lw=2)
  ax[1].plot(history.history['val_categorical_accuracy'], label='val accuracy', c='r', lw=2)
  ax[1].set_ylim(0,1)
  ax[1].hlines(0.5, 0, EPOCHS, color='y', ls=':', lw=4, label='Null Hypothesis\n(perfect 50% tag balance case)')
  ax[1].grid()
  ax[1].set_title('Training and Validation Acurracy over the Epochs')
  ax[1].legend()																																																																																																																																																																				

  plt.show()

In [ ]:
if DATASET_ITERATOR_APPROACH:
  # Print the raw values of the metrics, just in case we want to use them without re-running the notebook :O
  print(history.history['loss'])
  print(history.history['val_loss'])
  print(history.history['categorical_accuracy'])
  print(history.history['val_categorical_accuracy'])